
# AMLD Workshop Dry Run Notebook: Learn to Develop and Customize AI Workflows with Flows: A Hands-on Tutorial on Jarvis - an Open Source ChatGPT+(++)

## Introduction
Welcome to the dry run of the workshop! This notebook is designed for you to test the setup and ensure that everything runs smoothly before the actual workshop begins. Please follow the instructions below to execute the notebook successfully. If you encounter any issues, feel free to reach out to the workshop organizers for assistance.

You can contact us on our Discord server: [https://discord.gg/pFQTddUbjK](https://discord.gg/pFQTddUbjK)

## Prerequisites
Before proceeding, ensure you have the following installed:
- Python 3.10 or later
- Any setup which allows running a notebook (e.g., Jupyter Notebook, JupyterLab)


Please don't worry if you don't understand all the code in this notebook! The workshop will include explanations and discussions to help you understand and customize the AI workflows effectively.

## 1. Installing aiflows + Imports

In [1]:
!pip install aiflows==1.0.0

  Using cached aiflows-1.0.0-py3-none-any.whl.metadata (13 kB)
Using cached aiflows-1.0.0-py3-none-any.whl (75 kB)


In [2]:
###Some Useful Functions + imports

from IPython.core.magic import register_cell_magic
from aiflows.workers import run_dispatch_worker_thread
from aiflows.base_flows import AtomicFlow
from aiflows.messages import FlowMessage
from omegaconf import OmegaConf
import os
from aiflows.utils import colink_utils, serve_utils
from aiflows.utils import logging
logging.set_verbosity_info()

@register_cell_magic
def compile_and_writefile(line, cell):
    # Compile the code in the cell
    compiled_code = compile(cell, '<string>', 'exec')
    #check if all directories exist and create them if they don't   
    # Extract the directory path from the file path
    directory = os.path.dirname(line)

    # Check if the directory exists
    if not os.path.exists(directory):
        # Create the directory if it doesn't exist
        os.makedirs(directory)
        # Write the compiled code to a file
    with open(line, 'w') as f:
        f.write(cell)
        

def dict_to_yaml(dictionary, output_file):
    """
    Convert a dictionary to YAML using OmegaConf and write to a file.

    :param dictionary: Dictionary to convert.
    :type dictionary: dict
    :param output_file: Path to the output YAML file.
    :type output_file: str
    """
    # Convert dictionary to OmegaConf config object
    config = OmegaConf.create(dictionary)

    # Write the config object to the output YAML file
    OmegaConf.save(config, output_file)

/Users/nicolasbaldwin/opt/miniconda3/envs/mockenv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 2. Defining a Mock Flow

In [3]:
%%compile_and_writefile ReverseNumberFlowModule/ReverseNumberAtomicFlow.py

from aiflows.base_flows import AtomicFlow
from aiflows.messages import FlowMessage

class ReverseNumberAtomicFlow(AtomicFlow):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    # Customize the logic within this function as needed for your specific flow requirements.
    def run(self, input_message: FlowMessage):

        #Get data dictionary from input message
        input_data = input_message.data
        
        #get input number from data dictionary (int)
        input_number = input_data["number"]
        
        #TODO: reverse the input number (e.g. 1234 -> 4321)
        reversed_number = int(str(input_number)[::-1])
        
        #Create response dictionary
        response = {"reversed_number": reversed_number}
        
        #package ouput message to send back
            #This method packages `response` in a FlowMessage object 
            # containing the necessary metadata to send the message back
            # to the sender of the input message. 
        reply = self.package_output_message(
            input_message=input_message,
            response=response,
        )
        
        #send back reply
        self.send_message(
            reply
        )

In [4]:
default_config_reverse_number = \
{
    "name": "ReverseNumber",
    "description": "A flow that takes in a number and reverses it.",

    "_target_": "ReverseNumberFlowModule.ReverseNumberAtomicFlow.ReverseNumberAtomicFlow.instantiate_from_default_config",

    "input_interface": "number",
    "output_interface": "reversed_number",
}

dict_to_yaml(default_config_reverse_number, "ReverseNumberFlowModule/ReverseNumberAtomicFlow.yaml")

## 3. Starting a CoLink Server, Serving the Flow and getting an Instance

In [5]:
#Starting colink server
cl = colink_utils.start_colink_server()

In [6]:
#Serving the flow
serve_utils.serve_flow(
    cl=cl,
    flow_class_name="ReverseNumberFlowModule.ReverseNumberAtomicFlow.ReverseNumberAtomicFlow",
    flow_endpoint="reverse_number_atomic",
    singleton=True,
)

[2024-03-21 11:53:20,515][aiflows.utils.serve_utils:116][INFO] - Started serving ReverseNumberFlowModule.ReverseNumberAtomicFlow.ReverseNumberAtomicFlow at flows:reverse_number_atomic.
[2024-03-21 11:53:20,518][aiflows.utils.serve_utils:117][INFO] - dispatch_point: coflows_dispatch
[2024-03-21 11:53:20,520][aiflows.utils.serve_utils:118][INFO] - parallel_dispatch: False
[2024-03-21 11:53:20,521][aiflows.utils.serve_utils:119][INFO] - singleton: True



True

In [7]:
# Start a worker thread to handle incoming messages
run_dispatch_worker_thread(cl)

[2024-03-21 11:53:21,429][aiflows.workers.dispatch_worker:220][INFO] - Dispatch worker started in attached thread.
[2024-03-21 11:53:21,430][aiflows.workers.dispatch_worker:221][INFO] - dispatch_point: coflows_dispatch


In [8]:
#Getting an instance of the flow
proxy_reverse_number_flow = serve_utils.get_flow_instance(
    cl=cl,
    flow_endpoint="reverse_number_atomic",
    user_id="local",
)

[2024-03-21 11:53:22,409][aiflows.utils.serve_utils:336][INFO] - Mounted 4b368a0e-16e0-49f7-80d2-dce560c216db at flows:reverse_number_atomic:mounts:local:4b368a0e-16e0-49f7-80d2-dce560c216db


## 4. Running the Flow

In [9]:
input_data = {"number": 12345}

# Package your data in a Flow Message

## Option 1: Via the FlowMessage class
# input_message = FlowMessage(
#     data=input_data,
# )

## Option 2 (prefered): Via the package input message method
input_message = proxy_reverse_number_flow.package_input_message(input_data)

# Send a message to reverse number and ask to get an answer back in a future

future = proxy_reverse_number_flow.get_reply_future(input_message)

# Get the response from the future
#To get the response as a data dictionary
reply_data = future.get_data()

print("Data sent:\n",  input_data, "\n")
print("REPLY:\n", reply_data, "\n")


[2024-03-21 11:53:24,485][aiflows.workers.dispatch_worker:113][INFO] - 
~~~ Dispatch task ~~~
[2024-03-21 11:53:24,519][aiflows.workers.dispatch_worker:155][INFO] - flow_endpoint: reverse_number_atomic
[2024-03-21 11:53:24,520][aiflows.workers.dispatch_worker:156][INFO] - flow_id: 4b368a0e-16e0-49f7-80d2-dce560c216db
[2024-03-21 11:53:24,522][aiflows.workers.dispatch_worker:157][INFO] - owner_id: local
[2024-03-21 11:53:24,523][aiflows.workers.dispatch_worker:158][INFO] - message_paths: ['push_tasks:1b29bd19-a3ee-4b2d-8111-ba9a1d698593:msg']
[2024-03-21 11:53:24,523][aiflows.workers.dispatch_worker:159][INFO] - parallel_dispatch: False

[2024-03-21 11:53:24,558][aiflows.workers.dispatch_worker:182][INFO] - Input message source: Proxy_reverse_number_atomic


Data sent:
 {'number': 12345} 

REPLY:
 {'reversed_number': 54321} 



## Extra Todos
- Ensure you possess an **OpenAI API key** (check out [this link](https://platform.openai.com/docs/quickstart?context=python) on how to get one) or a key from a provider supported by litellm (check the list of supported providers [here]((https://docs.litellm.ai/docs/providers))). However, we highly recommend using an OpenAI key for this workshop. 

## Troubleshooting
If you encounter any errors or issues while running the notebook, please:

- Double-check that you have followed the setup instructions correctly.
- Review any error messages displayed in the notebook and contact us on our [Discord](https://discord.gg/pFQTddUbjK) server for assistance.

## You're ready for the workshop!

Once you have completed the dry run successfully, you're all set for the workshop! We look forward to seeing you there.